In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
import os
os.chdir('../../')

In [3]:
df = pd.read_excel('./documents/june_eighteen_var.xlsx').iloc[:,1::]

In [4]:
df

,Total Fertility Rate,Total Population,Seoul Population,Gyeonggi Province Population,Incheon City Population,Capital Region Population,Newborn Population,Death Population,Marriage Count,Divorce Count,...,Industrial Products,Rent,Public Services,Personal Services,Consumer Price Index.1,Interest Rate,USD Exchange Rate,International Oil Prices,Gross Domestic Product,Capital Region Population Density
0,1.58,41265113,9798542,5075449,1441131,16315122,69708,22924,36162,3191,...,2.3,4.7,2.3,4.5,30.117,9.70,890.53,19.66,102985.8,0.395373
1,1.58,41265113,9798542,5075449,1441131,16315122,73659,20258,36522,2997,...,2.2,4.5,2.3,4.5,30.293,9.70,886.39,13.23,102985.8,0.395373
2,1.58,41265113,9798542,5075449,1441131,16315122,48593,22144,38380,3465,...,2.2,4.5,2.1,5.2,30.353,9.70,884.37,10.40,102985.8,0.395373
3,1.58,41265113,9798542,5075449,1441131,16315122,47880,20892,36333,3501,...,1.3,4.3,2.6,5.1,30.323,9.70,885.60,13.34,102985.8,0.395373
4,1.58,41265113,9798542,5075449,1441131,16315122,47800,18495,37520,3377,...,1.7,4.0,2.4,5.1,30.501,9.70,887.21,14.25,102985.8,0.395373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,0.81,51671569,9558153,13512867,2937440,26008460,22356,25748,15739,8306,...,1.1,1.1,0.3,2.3,102.260,0.50,1143.98,73.95,2057448.0,0.503342
427,0.81,51669716,9550227,13530519,2938429,26019175,22287,25953,14720,8376,...,1.3,1.2,0.4,2.3,102.750,0.75,1160.34,68.50,2057448.0,0.503567
428,0.81,51667688,9542256,13542284,2941795,26026335,21893,25674,13733,8365,...,1.5,1.2,0.4,2.4,103.170,0.75,1169.54,75.03,2057448.0,0.503726
429,0.81,51662290,9532428,13549577,2945009,26027014,20741,27775,15203,7704,...,1.8,1.3,0.9,2.4,103.350,0.75,1182.82,83.57,2057448.0,0.503791


In [5]:
dnn = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
scaler = MinMaxScaler()
scale_data = scaler.fit_transform(df)

In [7]:
X = scale_data[:,0:21]
y = scale_data[:,21:22]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [9]:
class Losses:
    def __init__(self):
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
    def generator_loss(self, output, generated):
        entropy_loss = self.cross_entropy(tf.ones_like(output), output)
        population_loss = dnn(generated)

        total_loss = entropy_loss + population_loss
        return total_loss

    def discriminator_loss(self, true_output, fake_output):
        real_loss = self.cross_entropy(tf.ones_like(true_output), true_output)
        fake_loss = self.cross_entropy(tf.zeros_like(fake_output), fake_output)

        total_loss = real_loss + fake_loss
        return total_loss

In [10]:
EPOCHS = 100000
BATCH_SIZE = 32
noise_dim = 10
losses = Losses()

In [11]:
train_set = []
tmp = []
for i in range(len(X_train)-1):
    tmp.append(X_train[i:i+1])

    if (i % BATCH_SIZE == 0 and i != 0):
        train_set.append(tmp)
        tmp = []

In [12]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_shape=(10,), activation='relu'))
generator.add(tf.keras.layers.Dense(256, activation='relu'))
generator.add(tf.keras.layers.Dense(512, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='linear'))

generator.compile(optimizer='adam')

In [13]:
discriminator = tf.keras.Sequential()
discriminator.add(tf.keras.layers.Dense(256, input_shape=(21,), activation='relu'))
discriminator.add(tf.keras.layers.Dense(512, activation='relu'))
discriminator.add(tf.keras.layers.Dense(128, activation='relu'))
discriminator.add(tf.keras.layers.Dense(1, activation='sigmoid'))

discriminator.compile(optimizer='adam')

In [14]:
@tf.function
def train_step(data):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        generated = generator(noise, training=True)

        real_output = discriminator(data, training=True)
        fake_output = discriminator(generated, training=True)

        gen_loss = losses.generator_loss(fake_output, generated)
        dis_loss = losses.discriminator_loss(real_output, fake_output)
    
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator.optimizer.apply_gradients(zip(gen_grads, generator.trainable_variables))

    dis_grads = dis_tape.gradient(dis_loss, discriminator.trainable_variables)
    discriminator.optimizer.apply_gradients(zip(dis_grads, discriminator.trainable_variables))

In [15]:
def train(dataset, epochs):
    for eposh in range(epochs):
        for batch in dataset:
            train_step(batch[0])

In [16]:
train(train_set, EPOCHS)

In [17]:
discriminator(generator(tf.random.normal([1, 10])))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.09103263]], dtype=float32)>

In [18]:
discriminator(X)

<tf.Tensor: shape=(431, 1), dtype=float32, numpy=
array([[5.32680303e-07],
       [5.19363326e-04],
       [9.72279727e-01],
       [9.99952316e-01],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [9.99999523e-01],
       [9.48181391e-01],
       [5.23801595e-02],
       [3.63129377e-01],
       [2.34100390e-02],
       [9.17094827e-01],
       [9.91646528e-01],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [9.99908090e-01],
       [9.98446047e-01],
       [9.99647558e-01],
       [9.99999881e-01],
       [1.00000000e+00],
       [9.83525217e-01],
       [9.99999881e-01],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00],


In [19]:
discriminator.save('./model/discriminator.h5')

In [20]:
generator.save('./model/generator.h5')